In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors

In [ ]:
data = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/populationWide_bootstrap_kernel.csv')

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Step 1: Load data

# Step 2: Group the cell types
def group_cell_types(cell, position):
    if cell.startswith('B'):
        return 'B_from' if position == 'from' else 'B_to'
    elif cell.startswith('Mye'):
        return 'Mye_from' if position == 'from' else 'Mye_to'
    elif cell.startswith('Stromal'):
        return 'Stromal_from' if position == 'from' else 'Stromal_to'
    elif cell.startswith('Epi'):
        return 'Epi_from' if position == 'from' else 'Epi_to'
    elif cell.startswith('TNK'):
        return 'TNK_from' if position == 'from' else 'TNK_to'
    return cell

data['from_grouped'] = data['from'].apply(lambda x: group_cell_types(x, 'from'))
data['to_grouped'] = data['to'].apply(lambda x: group_cell_types(x, 'to'))

# Step 3: Summarize the data by counting unique pairs
grouped_data = data.groupby(['from_grouped', 'to_grouped']).size().reset_index(name='pair_count')

# Step 4: Prepare the nodes and their order
unique_nodes = ['B_from', 'B_to', 'Mye_from', 'Mye_to', 
                'Stromal_from', 'Stromal_to', 'Epi_from', 'Epi_to', 
                'TNK_from', 'TNK_to']
node_indices = {node: i for i, node in enumerate(unique_nodes)}

grouped_data['from_idx'] = grouped_data['from_grouped'].map(node_indices)
grouped_data['to_idx'] = grouped_data['to_grouped'].map(node_indices)

# Step 5: Define a color palette for the nodes with transparency
node_colors = ['rgba(255, 153, 153, 0.5)', 'rgba(255, 153, 153, 0.5)', 
               'rgba(102, 178, 255, 0.5)', 'rgba(102, 178, 255, 0.5)', 
               'rgba(153, 255, 153, 0.5)', 'rgba(153, 255, 153, 0.5)', 
               'rgba(255, 204, 153, 0.5)', 'rgba(255, 204, 153, 0.5)',
               'rgba(255, 153, 204, 0.5)', 'rgba(255, 153, 204, 0.5)']

# Assign colors to each link based on the source (from) node with transparency
link_colors = [node_colors[grouped_data['from_idx'][i]] for i in range(len(grouped_data))]

# Step 6: Create the Sankey diagram with transparency
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=unique_nodes,
        color=node_colors  # Transparent colors for nodes
    ),
    link=dict(
        source=grouped_data['from_idx'],
        target=grouped_data['to_idx'],
        value=grouped_data['pair_count'],  # Use pair count instead of frequency
        color=link_colors,  # Transparent colors for the edges matching the source (from) node
        line=dict(width=0.5)  # Narrower edges
    )
))

# Step 7: Customize the layout to make the figure narrower
fig.update_layout(
    title_text="Cell Type Transitions",
    font_size=10,
    width=600,  # Adjust this to make the figure narrower (default is 1000)
    height=600  # Adjust the height if necessary
)

# Step 8: Display the diagram
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Assuming 'data' is your DataFrame

# Step 2: Group the cell types
def group_cell_types(cell, position):
    if cell.startswith('B'):
        return 'B_from' if position == 'from' else 'B_to'
    elif cell.startswith('Mye'):
        return 'Mye_from' if position == 'from' else 'Mye_to'
    elif cell.startswith('Stromal'):
        return 'Stromal_from' if position == 'from' else 'Stromal_to'
    elif cell.startswith('Epi'):
        return 'Epi_from' if position == 'from' else 'Epi_to'
    elif cell.startswith('TNK'):
        return 'TNK_from' if position == 'from' else 'TNK_to'
    return cell

data['from_grouped'] = data['from'].apply(lambda x: group_cell_types(x, 'from'))
data['to_grouped'] = data['to'].apply(lambda x: group_cell_types(x, 'to'))

# Step 3: Summarize the data by counting unique pairs
grouped_data = data.groupby(['from_grouped', 'to_grouped']).size().reset_index(name='pair_count')

# Step 4: Filter out self-transitions and corresponding transitions (e.g., B_from -> B_to)
grouped_data = grouped_data[
    (grouped_data['from_grouped'] != grouped_data['to_grouped']) & 
    ~(
        (grouped_data['from_grouped'].str.contains('_from')) & 
        (grouped_data['to_grouped'].str.contains('_to')) &
        (grouped_data['from_grouped'].str.split('_').str[0] == grouped_data['to_grouped'].str.split('_').str[0])
    )
]

# Step 5: Prepare the nodes and their order
unique_nodes = ['B_from', 'B_to', 
                'Mye_from', 'Mye_to', 
                'Stromal_from', 'Stromal_to', 
                'Epi_from', 'Epi_to', 
                'TNK_from', 'TNK_to']

node_indices = {node: i for i, node in enumerate(unique_nodes)}

grouped_data['from_idx'] = grouped_data['from_grouped'].map(node_indices)
grouped_data['to_idx'] = grouped_data['to_grouped'].map(node_indices)

# Step 6: Define a color palette for the nodes with transparency
node_colors = ['rgba(255, 153, 153, 0.5)', 'rgba(255, 153, 153, 0.5)', 
               'rgba(102, 178, 255, 0.5)', 'rgba(102, 178, 255, 0.5)', 
               'rgba(153, 255, 153, 0.5)', 'rgba(153, 255, 153, 0.5)', 
               'rgba(255, 204, 153, 0.5)', 'rgba(255, 204, 153, 0.5)',
               'rgba(255, 153, 204, 0.5)', 'rgba(255, 153, 204, 0.5)']

# Assign colors to each link based on the source (from) node with transparency
link_colors = [node_colors[int(grouped_data.iloc[i]['from_idx'])] for i in range(len(grouped_data))]

# Step 7: Create the Sankey diagram with transparency
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=unique_nodes,
        color=node_colors  # Transparent colors for nodes
    ),
    link=dict(
        source=grouped_data['from_idx'],
        target=grouped_data['to_idx'],
        value=grouped_data['pair_count'],  # Use pair count instead of frequency
        color=link_colors,  # Transparent colors for the edges matching the source (from) node
        line=dict(width=0.5)  # Narrower edges
    )
))

# Step 8: Customize the layout to make the figure narrower
fig.update_layout(
    title_text="Cross Cell Type Transitions",
    font_size=10,
    width=600,  # Adjust this to make the figure narrower (default is 1000)
    height=600  # Adjust the height if necessary
)

# Step 9: Display the diagram
fig.show()


In [ ]:
data = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/populationWide_bootstrap_quantile.csv')

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Step 1: Load data

# Step 2: Group the cell types
def group_cell_types(cell, position):
    if cell.startswith('B'):
        return 'B_from' if position == 'from' else 'B_to'
    elif cell.startswith('Mye'):
        return 'Mye_from' if position == 'from' else 'Mye_to'
    elif cell.startswith('Stromal'):
        return 'Stromal_from' if position == 'from' else 'Stromal_to'
    elif cell.startswith('Epi'):
        return 'Epi_from' if position == 'from' else 'Epi_to'
    elif cell.startswith('TNK'):
        return 'TNK_from' if position == 'from' else 'TNK_to'
    return cell

data['from_grouped'] = data['from'].apply(lambda x: group_cell_types(x, 'from'))
data['to_grouped'] = data['to'].apply(lambda x: group_cell_types(x, 'to'))

# Step 3: Summarize the data by counting unique pairs
grouped_data = data.groupby(['from_grouped', 'to_grouped']).size().reset_index(name='pair_count')

# Step 4: Prepare the nodes and their order
unique_nodes = ['B_from', 'B_to', 'Mye_from', 'Mye_to', 
                'Stromal_from', 'Stromal_to', 'Epi_from', 'Epi_to', 
                'TNK_from', 'TNK_to']
node_indices = {node: i for i, node in enumerate(unique_nodes)}

grouped_data['from_idx'] = grouped_data['from_grouped'].map(node_indices)
grouped_data['to_idx'] = grouped_data['to_grouped'].map(node_indices)

# Step 5: Define a color palette for the nodes with transparency
node_colors = ['rgba(255, 153, 153, 0.5)', 'rgba(255, 153, 153, 0.5)', 
               'rgba(102, 178, 255, 0.5)', 'rgba(102, 178, 255, 0.5)', 
               'rgba(153, 255, 153, 0.5)', 'rgba(153, 255, 153, 0.5)', 
               'rgba(255, 204, 153, 0.5)', 'rgba(255, 204, 153, 0.5)',
               'rgba(255, 153, 204, 0.5)', 'rgba(255, 153, 204, 0.5)']

# Assign colors to each link based on the source (from) node with transparency
link_colors = [node_colors[grouped_data['from_idx'][i]] for i in range(len(grouped_data))]

# Step 6: Create the Sankey diagram with transparency
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=unique_nodes,
        color=node_colors  # Transparent colors for nodes
    ),
    link=dict(
        source=grouped_data['from_idx'],
        target=grouped_data['to_idx'],
        value=grouped_data['pair_count'],  # Use pair count instead of frequency
        color=link_colors,  # Transparent colors for the edges matching the source (from) node
        line=dict(width=0.5)  # Narrower edges
    )
))

# Step 7: Customize the layout to make the figure narrower
fig.update_layout(
    title_text="Cell Type Transitions",
    font_size=10,
    width=600,  # Adjust this to make the figure narrower (default is 1000)
    height=600  # Adjust the height if necessary
)

# Step 8: Display the diagram
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Assuming 'data' is your DataFrame

# Step 2: Group the cell types
def group_cell_types(cell, position):
    if cell.startswith('B'):
        return 'B_from' if position == 'from' else 'B_to'
    elif cell.startswith('Mye'):
        return 'Mye_from' if position == 'from' else 'Mye_to'
    elif cell.startswith('Stromal'):
        return 'Stromal_from' if position == 'from' else 'Stromal_to'
    elif cell.startswith('Epi'):
        return 'Epi_from' if position == 'from' else 'Epi_to'
    elif cell.startswith('TNK'):
        return 'TNK_from' if position == 'from' else 'TNK_to'
    return cell

data['from_grouped'] = data['from'].apply(lambda x: group_cell_types(x, 'from'))
data['to_grouped'] = data['to'].apply(lambda x: group_cell_types(x, 'to'))

# Step 3: Summarize the data by counting unique pairs
grouped_data = data.groupby(['from_grouped', 'to_grouped']).size().reset_index(name='pair_count')

# Step 4: Filter out self-transitions and corresponding transitions (e.g., B_from -> B_to)
grouped_data = grouped_data[
    (grouped_data['from_grouped'] != grouped_data['to_grouped']) & 
    ~(
        (grouped_data['from_grouped'].str.contains('_from')) & 
        (grouped_data['to_grouped'].str.contains('_to')) &
        (grouped_data['from_grouped'].str.split('_').str[0] == grouped_data['to_grouped'].str.split('_').str[0])
    )
]

# Step 5: Prepare the nodes and their order
unique_nodes = ['B_from', 'B_to', 
                'Mye_from', 'Mye_to', 
                'Stromal_from', 'Stromal_to', 
                'Epi_from', 'Epi_to', 
                'TNK_from', 'TNK_to']

node_indices = {node: i for i, node in enumerate(unique_nodes)}

grouped_data['from_idx'] = grouped_data['from_grouped'].map(node_indices)
grouped_data['to_idx'] = grouped_data['to_grouped'].map(node_indices)

# Step 6: Define a color palette for the nodes with transparency
node_colors = ['rgba(255, 153, 153, 0.5)', 'rgba(255, 153, 153, 0.5)', 
               'rgba(102, 178, 255, 0.5)', 'rgba(102, 178, 255, 0.5)', 
               'rgba(153, 255, 153, 0.5)', 'rgba(153, 255, 153, 0.5)', 
               'rgba(255, 204, 153, 0.5)', 'rgba(255, 204, 153, 0.5)',
               'rgba(255, 153, 204, 0.5)', 'rgba(255, 153, 204, 0.5)']

# Assign colors to each link based on the source (from) node with transparency
link_colors = [node_colors[int(grouped_data.iloc[i]['from_idx'])] for i in range(len(grouped_data))]

# Step 7: Create the Sankey diagram with transparency
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=unique_nodes,
        color=node_colors  # Transparent colors for nodes
    ),
    link=dict(
        source=grouped_data['from_idx'],
        target=grouped_data['to_idx'],
        value=grouped_data['pair_count'],  # Use pair count instead of frequency
        color=link_colors,  # Transparent colors for the edges matching the source (from) node
        line=dict(width=0.5)  # Narrower edges
    )
))

# Step 8: Customize the layout to make the figure narrower
fig.update_layout(
    title_text="Cross Cell Type Transitions",
    font_size=10,
    width=600,  # Adjust this to make the figure narrower (default is 1000)
    height=600  # Adjust the height if necessary
)

# Step 9: Display the diagram
fig.show()


In [ ]:
data_quantile = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/populationWide_bootstrap_quantile.csv')
data_kernel = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/populationWide_bootstrap_kernel.csv')

In [ ]:
# create a column for edge names
data_quantile['edge'] = data_quantile['from'] + ' -> ' + data_quantile['to']
data_kernel['edge'] = data_kernel['from'] + ' -> ' + data_kernel['to']
data_kernel

In [ ]:
# plot the venn diagram of the two datasets
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from matplotlib import pyplot as plt

# create a venn diagram
plt.figure(figsize=(6, 6))
venn2([set(data_quantile['edge']), set(data_kernel['edge'])], set_labels=('Quantile', 'Kernel'))
plt.title('Quantile vs Kernel')    
plt.show()


In [ ]:
# print the overlappeing edges
quantile_edges = set(data_quantile['edge'])
kernel_edges = set(data_kernel['edge'])
common_edges = quantile_edges.intersection(kernel_edges)
common_edges


# overlap with the LR_CIT

In [ ]:
# change the edge names 'B' -> 'PlasmaB', 'mye' -> 'Myeloid', 'stromal' -> 'Stroma'
def change_edge_name(edge):
    edge = edge.replace('B', 'PlasmaB')
    edge = edge.replace('Mye', 'Myeloid')
    edge = edge.replace('Stromal', 'Stroma')
    return edge

data_quantile['edge'] = data_quantile['edge'].apply(lambda x: change_edge_name(x))
data_kernel['edge'] = data_kernel['edge'].apply(lambda x: change_edge_name(x))

In [ ]:
data_LR = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/LR_passed_search_permutation.csv')

In [ ]:
data_LR

In [ ]:
data_LR['edge'] = data_LR['from_GEM(X)'] + ' -> ' + data_LR['to_GEM(Y)']

In [ ]:
# only keep the edges that are in the data_quantile or data_kernel
data_LR_kernel = data_LR[data_LR['edge'].isin(data_kernel['edge'])]
data_LR_quantile = data_LR[data_LR['edge'].isin(data_quantile['edge'])]

In [ ]:
data_LR_quantile

In [ ]:
data_LR_kernel

In [ ]:
# save the data
data_LR_kernel.to_csv('/home/qiuaodon/Desktop/project_data_new/LR_passed_search_permutation_kernel.csv', index=False)
data_LR_quantile.to_csv('/home/qiuaodon/Desktop/project_data_new/LR_passed_search_permutation_quantile.csv', index=False)